In [ ]:
!pip install -U gdown
!pip install wikipedia-api

import pandas as pd
import wikipediaapi
from tqdm.notebook import tqdm
import gdown
import pandas as pd
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
from tqdm import tqdm
import json
import urllib.parse
import time

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=e9eb0039260233c025fcdcc2fe4a244f458c92dfa46d01e63485826d626ba129
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built wikipedia-api


# Cleaning Json

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import scipy
from sklearn.metrics.pairwise import cosine_similarity
import joblib

---

In [ ]:
file_id = "1i-NpiN5v5yrXI3s2IxHqBiSlg0_URD-c"
url = f"https://drive.google.com/uc?id={file_id}"

output = "dataset.json"

gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1i-NpiN5v5yrXI3s2IxHqBiSlg0_URD-c
To: /content/dataset.json
100%|██████████| 168k/168k [00:00<00:00, 5.04MB/s]


'dataset.json'

In [ ]:
with open('dataset.json', 'r') as f:
    data = json.load(f)

filtered_data =[]
for item in data:
  filtered_data.append({
      'latin': item.get('latin'),
      'family': item.get('family'),
      'category': item.get('category'),
      'climate': item.get('climate'),
      'ideallight': item.get('ideallight'),
      'toleratedlight': item.get('toleratedlight'),
      'watering': item.get('watering'),
      'insects': item.get('insects') if isinstance(item.get('insects'), list) else [item.get('insects')],
      'use': item.get('use') if isinstance(item.get('use'), list) else [item.get('use')],
      'tempmax_celsius': item.get('tempmax', {}).get('celsius'),
      'tempmin_celsius': item.get('tempmin', {}).get('celsius'),
  })

In [ ]:
df = pd.DataFrame(filtered_data)
df.head()

,latin,family,category,climate,ideallight,toleratedlight,watering,insects,use,tempmax_celsius,tempmin_celsius
0,Aeschynanthus lobianus,Gesneriaceae,Hanging,Tropical,Bright light,Direct sunlight,Keep moist between watering. Can be a bit dry ...,"[Mealy bug, Aphid, Thrips]","[Hanging, Flower, Tertiary]",32,14
1,Adiantum raddianum,Polypodiaceae,Fern,Tropical,Bright light,Diffused,Keep moist between watering. Must not be dry b...,"[Mealy bug, Aphid, Snail]","[Potted plant, Ground cover, Table top]",30,12
2,Aechmea fatsiata,Bromeliaceae,Bromeliad,Tropical humid,Bright light,Diffused,Water when soil is half dry. Change water in t...,[N/A],"[Flower, Table top, Tertiary]",30,12
3,Agave angustilolia Marginata,Amaryllidaceae,Cactus And Succulent,Tropical,6 or more hours of direct sunlight per day.,Direct sunlight.,Water only when the soil is dry. Must be dry b...,"[Scale, Mealy bug]","[Potted plant, Primary, Secondary]",35,5
4,Aechmea ramosa,Bromeliaceae,Bromeliad,Subtropical,Bright light,Diffused,Water when soil is half dry. Change water in t...,[N/A],"[Flower, Table top, Primary]",30,12


In [ ]:
df.isnull().sum()

,0
latin,0
family,0
category,0
climate,0
ideallight,0
toleratedlight,0
watering,0
insects,0
use,0
tempmax_celsius,0


In [ ]:
df['insects'] = df['insects'].apply(lambda x: ', '.join(x) if isinstance(x, list) else str(x))
df['use'] = df['use'].apply(lambda x: ', '.join(x) if isinstance(x, list) else str(x))

In [ ]:
df['temp_avg'] = df[['tempmin_celsius', 'tempmax_celsius']].mean(axis=1)

In [ ]:

df['combined'] = df['climate'] + ' ' + df['ideallight'] + ' ' + df['toleratedlight'] + ' ' + df['watering'] + ' ' + df['insects'] + ' ' + df['use']

tfidf = TfidfVectorizer()
X_text = tfidf.fit_transform(df['combined'])

X_temp = df[['temp_avg']].fillna(df['temp_avg'].mean())
scaler = StandardScaler()
X_temp_scaled = scaler.fit_transform(X_temp)

X = scipy.sparse.hstack([X_text, X_temp_scaled])

In [ ]:
def recommend_plant(user_text, user_temp, top_n=5):
    user_combined = user_text
    user_vector = tfidf.transform([user_combined])
    user_temp_scaled = scaler.transform([[user_temp]])
    user_input = scipy.sparse.hstack([user_vector, user_temp_scaled])

    similarities = cosine_similarity(user_input, X).flatten()
    indices = similarities.argsort()[-top_n:][::-1]

    return df.iloc[indices][['latin', 'category', 'climate', 'temp_avg']]


In [ ]:
joblib.dump(tfidf, 'tfidf.pkl')
joblib.dump(scaler, 'scaler.pkl')
df.to_csv('cleaned_plants.csv', index=False)

# Cleaning CSV

## **📥 Load Data**

In [ ]:
# Unduh dataset dari Google Drive dan langsung baca sebagai DataFrame
file_id_csv = '1nYJFILpf7PAfSSaKrv5FJK9FYlTZ4pmO'
url_csv = f"https://drive.google.com/uc?id={file_id_csv}"

# Download file dan baca ke pandas
gdown.download(url_csv, 'dataset.csv', quiet=False)
df = pd.read_csv('dataset.csv', encoding='ISO-8859-1')
df.head(5)

Downloading...
From: https://drive.google.com/uc?id=1nYJFILpf7PAfSSaKrv5FJK9FYlTZ4pmO
To: /content/dataset.csv
100%|██████████| 43.7k/43.7k [00:00<00:00, 42.1MB/s]


,Plant Name,Growth,Soil,Sunlight,Watering,Fertilization Type
0,Aloe Vera,slow,sandy,indirect sunlight,Water weekly,Balanced
1,Basil,fast,well-drained,full sunlight,Keep soil evenly moist,Organic
2,Snake Plant,slow,well-drained,indirect sunlight,Water when soil is dry,No
3,Lavender,moderate,sandy,full sunlight,Let soil dry between watering,No
4,Cactus,slow,sandy,full sunlight,Let soil dry between watering,Low-nitrogen


## **📤 Take & Show Family**

### **Via wiki**

In [ ]:
df.duplicated().sum()

np.int64(4)

In [ ]:
tqdm.pandas()

def make_family_fetcher(limit=None):
    counter = {'count': 0}

    def get_plant_family_from_wikipedia(plant_name):
        if limit is not None and counter['count'] >= limit:
            return "Skipped"

        session = requests.Session()

        def fetch_family_from_url(url):
            try:
                res = session.get(url, timeout=10)
                if res.status_code != 200:
                    return "Not Found"

                soup = BeautifulSoup(res.content, 'html.parser')

                # 1. Coba dari infobox
                infobox = soup.find("table", {"class": lambda x: x and "infobox" in x})
                if infobox:
                    for row in infobox.find_all("tr"):
                        header = row.find("th")
                        data = row.find("td")
                        if header and data and 'family' in header.text.lower():
                            family_text = data.get_text(separator=" ", strip=True)
                            if not family_text.lower().startswith('family'):
                                return family_text

                # 2. Coba dari taxonrow
                for row in soup.find_all("tr", class_="taxonrow"):
                    cells = row.find_all("td")
                    if len(cells) == 2 and "family" in cells[0].text.lower():
                        family_link = cells[1].find("a")
                        if family_link:
                            return family_link.text.strip()
                        else:
                            return cells[1].text.strip()

                # 3. Heading id="Plants" → cari <ul> setelahnya
                headings = soup.find_all("span", {"id": "Plants"})
                for heading in headings:
                    parent_h2 = heading.find_parent("h2")
                    if parent_h2:
                        next_node = parent_h2
                        while next_node:
                            next_node = next_node.find_next_sibling()
                            if next_node and next_node.name == "ul":
                                for li in next_node.find_all("li"):
                                    if "family" in li.text.lower():
                                        for link in li.find_all("a", href=True):
                                            if "/wiki/" in link["href"]:
                                                return link.text.strip()
                                break

                # 4. Cari di <ul> umum
                for ul in soup.find_all("ul", recursive=True):
                    for li in ul.find_all("li", recursive=False):
                        if "family" in li.text.lower():
                            link = li.find("a", href=True)
                            if link and "/wiki/" in link["href"]:
                                return link.text.strip()

                # 5. Ekstrak dari paragraf pertama
                for p in soup.find_all("p"):
                  lower_text = p.get_text().lower()
                  if "family" in lower_text:
                      found_family = False
                      for i, node in enumerate(p.contents):
                          # Jika ditemukan 'family' dalam string
                          if isinstance(node, NavigableString) and "family" in node.lower():
                              found_family = True
                          elif found_family:
                              # Cari tag <a> setelah kata "family"
                              if isinstance(node, Tag) and node.name == "a":
                                  family_text = node.get_text(strip=True)
                                  # Validasi hasil (hindari simbol IPA, dsb)
                                  if family_text and len(family_text) > 2 and "/" not in family_text:
                                      return family_text
                                  else:
                                      break  # hentikan jika <a> tidak valid

            except Exception:
                return "Not Found"

            return "Not Found"

        # Step 1: akses langsung halaman
        direct_url = f"https://en.wikipedia.org/wiki/{plant_name.replace(' ', '_')}"
        family = fetch_family_from_url(direct_url)
        if family != "Not Found":
            counter['count'] += 1
            return family

        # Step 2: hasil pencarian
        search_url = f"https://en.wikipedia.org/w/index.php?search={urllib.parse.quote(plant_name)}"
        try:
            search_response = session.get(search_url, timeout=10)
            soup = BeautifulSoup(search_response.content, 'html.parser')

            # Halaman hasil pencarian yang langsung mengandung info
            if soup.find("table", {"class": "infobox"}) or soup.find_all("tr", class_="taxonrow"):
                family = fetch_family_from_url(search_response.url)
                counter['count'] += 1
                return family

            # Hasil pertama
            first_result = soup.find("div", class_="mw-search-result-heading")
            if first_result:
                href = first_result.find("a")["href"]
                top_result_url = urllib.parse.urljoin("https://en.wikipedia.org", href)
                family = fetch_family_from_url(top_result_url)
                counter['count'] += 1
                return family

        except Exception:
            return "Not Found"

        return "Not Found"

    return get_plant_family_from_wikipedia

# Buat fetcher terbatas
fetch_limited = make_family_fetcher(limit=None)

# Jalankan
df_temp = df.assign(Family=df['Plant Name'].progress_apply(fetch_limited))
df_temp.head(30)

100%|██████████| 596/596 [01:46<00:00,  5.59it/s]


,Plant Name,Growth,Soil,Sunlight,Watering,Fertilization Type,Family
0,Aloe Vera,slow,sandy,indirect sunlight,Water weekly,Balanced,Asphodelaceae
1,Basil,fast,well-drained,full sunlight,Keep soil evenly moist,Organic,Lamiaceae
2,Snake Plant,slow,well-drained,indirect sunlight,Water when soil is dry,No,Not Found
3,Lavender,moderate,sandy,full sunlight,Let soil dry between watering,No,Lamiaceae
4,Cactus,slow,sandy,full sunlight,Let soil dry between watering,Low-nitrogen,Cactaceae
5,Rosemary,moderate,well-drained,full sunlight,Let soil dry between watering,No,Lamiaceae
6,Mint,fast,well-drained,partial sunlight,Keep soil consistently moist,Balanced,Lamiaceae
7,Thyme,moderate,sandy,full sunlight,Let soil dry between watering,Organic,Lamiaceae
8,Peace Lily,moderate,well-drained,indirect sunlight,Water when soil feels dry,Balanced,Araceae
9,Spider Plant,moderate,loamy,indirect sunlight,Keep soil slightly moist,Balanced,Chlorophytum


## **🔍 Check not Found**

In [ ]:
# Hitung jumlah Family yang "Not Found"
not_found_count = df_temp[df_temp['Family'] == 'Not Found'].shape[0]
print(f"Jumlah tanaman dengan family 'Not Found': {not_found_count}")

Jumlah tanaman dengan family 'Not Found': 395


## **🪢 Merge to Dataset**

In [ ]:
df_temp.to_csv("plantsNameNFamily.csv", index=False)

In [ ]:
df_temp.head(5)

,Plant Name,Growth,Soil,Sunlight,Watering,Fertilization Type,Family
0,Aloe Vera,slow,sandy,indirect sunlight,Water weekly,Balanced,Asphodelaceae
1,Basil,fast,well-drained,full sunlight,Keep soil evenly moist,Organic,Lamiaceae
2,Snake Plant,slow,well-drained,indirect sunlight,Water when soil is dry,No,Not Found
3,Lavender,moderate,sandy,full sunlight,Let soil dry between watering,No,Lamiaceae
4,Cactus,slow,sandy,full sunlight,Let soil dry between watering,Low-nitrogen,Cactaceae


In [ ]:
print(", ".join(df_temp.columns))

Plant Name, Growth, Soil, Sunlight, Watering, Fertilization Type, Family
